# Homework 9: Getting Familiar with NASA Polynomials
## Due Date:  Tuesday, November 7th at 11:59 PM

Read the NASA Polynomial dataset in raw format and parse and store the data into an .xml file.

### Review of the NASA Polynomials
You can find the NASA Polynomial file in `thermo.txt`.

You can find some details on the NASA Polynomials [at this site](http://combustion.berkeley.edu/gri_mech/data/nasa_plnm.html) in addition to the Lecture 16 notes.


The NASA polynomials for specie $i$ have the form:
$$
    \frac{C_{p,i}}{R}= a_{i1} + a_{i2} T + a_{i3} T^2 + a_{i4} T^3 + a_{i5} T^4
$$

$$
    \frac{H_{i}}{RT} = a_{i1} + a_{i2} \frac{T}{2} + a_{i3} \frac{T^2}{3} + a_{i4} \frac{T^3}{4} + a_{i5} \frac{T^4}{5} + \frac{a_{i6}}{T}
$$

$$
    \frac{S_{i}}{R}  = a_{i1} \ln(T) + a_{i2} T + a_{i3} \frac{T^2}{2} + a_{i4} \frac{T^3}{3} + a_{i5} \frac{T^4}{4} + a_{i7}
$$

where $a_{i1}$, $a_{i2}$, $a_{i3}$, $a_{i4}$, $a_{i5}$, $a_{i6}$, and $a_{i7}$ are the numerical coefficients supplied in NASA thermodynamic files. 

### Some Notes on `thermo.txt`
The first 7 numbers starting on the second line of each species entry (five of the second line and the first two of the third line) are the seven coefficients ($a_{i1}$ through $a_{i7}$, respectively) for the high-temperature range (above 1000 K, the upper boundary is specified on the first line of the species entry). 

The next seven numbers are the coefficients ($a_{i1}$ through $a_{i7}$, respectively) for the low-temperature range (below 1000 K, the lower boundary is specified on the first line of the species entry).

### Additional Specifications
Your final .xml file should contain the following specifications:

1. A `speciesArray` field that contains a space-separated list of all of the species present in the file.
2. Each species contains a `species` field with a `name` attribute as the species name.

    1. For each temperature range, use a sub-field with the minimum and maximum temperature as attributes.
    2. `floatArray` field that contains comma-separated string of each coefficient.
    
You can reference the `example_thermo.xml` file for an example .xml output.

**Hint**: First parse the file into a Python dictionary. 

In [1]:
import re
from copy import deepcopy
import xml.etree.ElementTree as ET

#read the file
file = open('thermo.txt','r')

#read in each line
lines = file.readlines()

#close the file
file.close()

#dictionary
species_info = {}

#iterate over lines
for i,line in enumerate(lines):
    #spilt the line
    strings = line.split()
    
    #stop if at the end of the file
    if strings[0]=="END":
        break  
    
    #find line with species
    if strings[-1] == '1':
        #print(strings)
        #get species name
        specie = strings[0]
        #print('specie: ', strings[0])
        #get the low temp min and max
        low_min = strings[-4]
        low_max = 1000.000
        
        #get the high temp min and max
        high_min = 1000.000
        high_max = strings[-3] 
        
    if strings[-1] == '2': 
        #spilt the line by number as the numbers are not broken up by spaces in txt file
        strings = re.findall(r"[+\-]?(?:0|[1-9]\d*)(?:\.\d*)?(?:[eE][+\-]?\d+)?", line)
        #get first 5 high coefs
        high_coeffs = []
        high_coeffs.extend(strings[0:-1])
        #print(strings[0:-1])
        
    if strings[-1] == '3': 
        #spilt the line by number as the numbers are not broken up by spaces in txt file
        strings = re.findall(r"[+\-]?(?:0|[1-9]\d*)(?:\.\d*)?(?:[eE][+\-]?\d+)?", line)
        #first two are high coeefs
        high_coeffs.extend(strings[0:2])
        
        #remaining are low coeffs
        low_coeffs = []
        low_coeffs.extend(strings[2:-1])
        
    if strings[-1] == '4': 
        #spilt the line by number as the numbers are not broken up by spaces in txt file
        strings = re.findall(r"[+\-]?(?:0|[1-9]\d*)(?:\.\d*)?(?:[eE][+\-]?\d+)?", line)
        # get low coefs
        low_coeffs.extend(strings[0:-1])
        #print("numbers in 4:",strings[0:-1])

        #Add to dictionary
        species_info[specie]={'l':{},'h':{}}
        species_info[specie]['l']['Tmax'] = low_max
        species_info[specie]['l']['Tmin'] = low_min
        species_info[specie]['l']['coeffs'] = low_coeffs
        species_info[specie]['h']['Tmax'] = high_max
        species_info[specie]['h']['Tmin'] = high_min
        species_info[specie]['h']['coeffs'] = high_coeffs
        
        #print everything
        #print('specie: ',specie)
        #print('low min: ',low_min)
        #print('low max: ',low_max)
        #print("low",low_coeffs)
        #print('high min: ',high_min)
        #print('high max: ',high_max)
        #print("high",high_coeffs)
        
#Print out dict to test

'''for i,k in enumerate(species_info):
    print(k)
    print(species_info[k]['h']['Tmax'])
    print(species_info[k]['h']['Tmin'])
    print(species_info[k]['h']['coeffs'])
    print(species_info[k]['l']['Tmax'])
    print(species_info[k]['l']['Tmin'])
    print(species_info[k]['l']['coeffs'])'''

low ['3.16826710E+00', '-3.27931884E-03', '6.64306396E-06', '-6.12806624E-09', '2.11265971E-12', '2.91222592E+04', '2.05193346E+00']
low ['3.78245636E+00', '-2.99673416E-03', '9.84730201E-06', '-9.68129509E-09', '3.24372837E-12', '-1.06394356E+03', '3.65767573E+00']
low ['2.50000000E+00', '7.05332819E-13', '-1.99591964E-15', '2.30081632E-18', '-9.27732332E-22', '2.54736599E+04', '-4.46682853E-01']
low ['2.34433112E+00', '7.98052075E-03', '-1.94781510E-05', '2.01572094E-08', '-7.37611761E-12', '-9.17935173E+02', '6.83010238E-01']
low ['3.99201543E+00', '-2.40131752E-03', '4.61793841E-06', '-3.88113333E-09', '1.36411470E-12', '3.61508056E+03', '-1.03925458E-01']
low ['4.19864056E+00', '-2.03643410E-03', '6.52040211E-06', '-5.48797062E-09', '1.77197817E-12', '-3.02937267E+04', '-8.49032208E-01']
low ['4.30179801E+00', '-4.74912051E-03', '2.11582891E-05', '-2.42763894E-08', '9.29225124E-12', '2.94808040E+02', '3.71666245E+00']
low ['4.27611269E+00', '-5.42822417E-04', '1.67335701E-05', '-2

"for i,k in enumerate(species_info):\n    print(k)\n    print(species_info[k]['h']['Tmax'])\n    print(species_info[k]['h']['Tmin'])\n    print(species_info[k]['h']['coeffs'])\n    print(species_info[k]['l']['Tmax'])\n    print(species_info[k]['l']['Tmin'])\n    print(species_info[k]['l']['coeffs'])"

In [5]:
#Create a xml from Dictionary
from copy import deepcopy
import xml.etree.ElementTree as ET

#Get the xml structure from file
tree = ET.parse('example_thermo.xml')
root = tree.getroot()

#Create species array element
speciesArray = deepcopy(root.find('phase').find('speciesArray'))

#Create string of all species
species_list = ''
for i,k in enumerate(species_info):
    species_list += ' ' + str(k)

#Add species string to copied species array element
speciesArray.text = species_list

#Delete old species array element
example_speciesArray= root.find('phase').find('speciesArray')
root.find('phase').remove(example_speciesArray)

#Append new species array eleemnt
root.find('phase').append(speciesArray)

#Make copy of a specie element to build elements off of
specie_temp = deepcopy(root.find('speciesData').find('species'))

#Delete old species entries 
example_species = root.find('speciesData').findall('species')
for specie in example_species:
    root.find('speciesData').remove(specie)

#create species from dict
for i,k in enumerate(species_info):
    
    #create element from template
    new_specie = deepcopy(specie_temp)

    #set name property to name of specie
    new_specie.set('name',str(k))
    
    #get NASA elements
    thermo = new_specie.find('thermo')
    NASA = thermo.findall('NASA')
    
    #create lower temp NASA element
    NASA[0].set("Tmax",str(species_info[k]['l']['Tmax']))
    NASA[0].set("Tmin",str(species_info[k]['l']['Tmin']))
    
    #get coeefcients string
    low_coeffs =",".join(species_info[k]['l']['coeffs'])
    NASA[0].find('floatArray').text = low_coeffs
    
    #create higher temp NASA element
    NASA[1].set("Tmax",species_info[k]['h']['Tmax'])
    NASA[1].set("Tmin",species_info[k]['h']['Tmin'])
    
    #get coeefcients string
    high_coeffs =",".join(species_info[k]['h']['coeffs'])
    NASA[1].find('floatArray').text = high_coeffs

    #add species to root
    root.find('speciesData').append(new_specie)
    
#create xml
tree.write('thermo.xml')

TypeError: cannot serialize 1000.0 (type float)

In [ ]:
def parse_dict_to_xml(species_dict):
    from copy import deepcopy
    import xml.etree.ElementTree as ET
    #use the template file
    tree = ET.parse('example_thermo.xml')
    root = tree.getroot()

    #get a node of species
    specie = deepcopy(root.find('speciesData').find('species'))

    #remove all the species from the parent
    old_species = root.find('speciesData').findall('species')

    for item in old_species:
        root.find('speciesData').remove(item)

    for index,key in enumerate(species_dict):

        temp = deepcopy(specie)
        temp.set('name',str(key))

        #species_dict[key]['low']['coefs']

        NASA = temp.find('thermo').findall('NASA')

        NASA[0].find('floatArray').text = ', '.join(species_dict[key]['low']['coefs'])
        NASA[0].set("Tmax",species_dict[key]['low']['tmax'])
        NASA[0].set("Tmin",species_dict[key]['low']['tmin'])

        NASA[1].find('floatArray').text = ', '.join(species_dict[key]['high']['coefs'])
        NASA[1].set("Tmax",species_dict[key]['high']['tmax'])
        NASA[1].set("Tmin",species_dict[key]['high']['tmin'])

        root.find('speciesData').append(temp)

    #creating my SpeciesArray
    for item in root.find('phase'):
        specie_names = [value for value in species_dict]
        item.text = " ".join(specie_names)

    return tree